In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 18.1 MB/s eta 0:00:00


In [3]:
%cd /content/drive/MyDrive/Liver Vessel/nnunet

/content/drive/MyDrive/Liver Vessel/nnunet


In [4]:
import SimpleITK as sitk
import numpy as np
import os
import glob
import pathlib
import warnings
from scipy.ndimage import distance_transform_edt # For centerline calculation
from skimage.morphology import skeletonize       # For centerline calculation
from tqdm import tqdm                            # For progress bar
import re                                        # For extracting number from filename

# --------------------------------------------------------------------------------------
# Centerline Map Generation Code (Provided by User)
# --------------------------------------------------------------------------------------
# --- Core routines ---
def F_mapping(d: np.ndarray) -> np.ndarray:
    """Paper’s mapping F(d) = 1 / log(e + d) (Eq-9)."""
    # Avoid division by zero or log(e)=1 issues if d is very small near 0
    # Add a small epsilon inside log, although log(e+d) should be safe for d>=0
    # Adding epsilon to d itself to prevent log(e) = 1 if d=0
    # Or handle d=0 separately: F(0) = 1/log(e) = 1
    mapped = np.zeros_like(d, dtype=np.float32)
    non_zero_d = d > 1e-7 # Check where d is significantly > 0
    zero_d = ~non_zero_d

    mapped[non_zero_d] = 1.0 / np.log(np.e + d[non_zero_d])
    mapped[zero_d] = 1.0 # F(0) = 1/log(e) = 1

    # Alternative direct calculation (might be slightly less safe for d near 0):
    # return 1.0 / np.log(np.e + d)
    return mapped

def normalise01(arr: np.ndarray) -> np.ndarray:
    """Normalizes array to [0, 1] range."""
    vmin, vmax = arr.min(), arr.max()
    diff = vmax - vmin
    if diff < 1e-7: # Avoid division by zero if the array is constant
        # If constant and non-zero, map to 1? Or 0?
        # Based on F(0)=1, a constant distance might imply skeleton = mask.
        # Let's return 1 for non-zero constant arrays, 0 for zero constant arrays.
        if vmax > 1e-7:
            return np.ones_like(arr, dtype=np.float32)
        else:
            return np.zeros_like(arr, dtype=np.float32)
    else:
        return ((arr - vmin) / diff).astype(np.float32)

def make_centerline_map(mask: np.ndarray, spacing: tuple[float, float, float]) -> np.ndarray:
    """
    Generates the centerline distance map based on Eq. 8 & 9.
    Assumes mask has 1 for vessel, 0 for background.
    Spacing should be provided in (z, y, x) order matching the NumPy array dimensions.
    """
    mask_bool = mask.astype(bool)
    if not mask_bool.any(): # Handle empty mask
        return np.zeros_like(mask, dtype=np.float32)

    # Use skimage.morphology.skeletonize for 3D skeletonization
    # Ensure mask is boolean
    skel = skeletonize(mask_bool)

    # Check if skeletonization yielded any results
    if not skel.any():
        warnings.warn("Skeletonization resulted in an empty set. Check input mask. Returning zero map for centerline.")
        return np.zeros_like(mask, dtype=np.float32)

    # Calculate Euclidean Distance Transform (EDT) to the nearest *skeleton* voxel.
    # distance_transform_edt computes distance to the nearest background (0) pixel.
    # So we need the EDT of the *inverted* skeleton mask (~skel).
    # The 'sampling' argument requires spacing corresponding to array axes (z, y, x).
    dist = distance_transform_edt(~skel, sampling=spacing)

    # Apply the mapping function F(d)
    dist_map = F_mapping(dist)

    # Create the output map: only assign values *inside* the original vessel mask
    out = np.zeros_like(dist_map, dtype=np.float32)
    out[mask_bool] = dist_map[mask_bool] # Apply map only within the original mask

    # Normalize the result *within the mask* to [0, 1]
    masked_values = out[mask_bool]
    if masked_values.size > 0:
        vmin, vmax = masked_values.min(), masked_values.max()
        diff = vmax - vmin
        if diff < 1e-7:
            # If constant, map non-zero values to 1, zero remains 0
            out[mask_bool] = 1.0 if vmax > 1e-7 else 0.0
        else:
            out[mask_bool] = ((masked_values - vmin) / diff)

    # Ensure correct type
    return out.astype(np.float32)
# --------------------------------------------------------------------------------------
# End of Centerline Map Generation Code
# --------------------------------------------------------------------------------------


# --- Configuration ---
# Input folder containing nnU-Net predictions for the TEST set (Task 2 format assumed)
# These files are named like 'hepaticvessel_000XX.nii.gz'
input_pred_folder = "data/VEELATestPredPP!_1.1.1"

# Base output folder for submission
output_base_folder = "v1.1.1/Results"

# Specific task folders within the output base folder
task1_folder = os.path.join(output_base_folder, "Task01") # For Segmentation results (Centerline Map)
task2_folder = os.path.join(output_base_folder, "Task02") # For Classification results

# --- Create Output Directories ---
# Create the base folder and the task subfolders
# exist_ok=True prevents an error if the folders already exist
os.makedirs(task1_folder, exist_ok=True)
os.makedirs(task2_folder, exist_ok=True)
print(f"Output folders created/ensured:")
print(f"- Task 1: {os.path.abspath(task1_folder)}")
print(f"- Task 2: {os.path.abspath(task2_folder)}")

# --- Find Input Prediction Files ---
# Find all .nii.gz files starting with 'hepaticvessel_' in the input folder
input_files = sorted(glob.glob(os.path.join(input_pred_folder, "hepaticvessel_*.nii.gz")))

# Check if any files were found
if not input_files:
    print(f"\nError: No 'hepaticvessel_*.nii.gz' files found in '{input_pred_folder}'.")
    print("Please ensure the 'input_pred_folder' path is correct and contains the prediction files.")
    exit() # Stop the script if no files are found

print(f"\nFound {len(input_files)} prediction files to process.")

# --- Process Each File ---
print("Starting conversion process...")
for input_filepath in tqdm(input_files, desc="Processing Test Predictions"):
    filename = os.path.basename(input_filepath)

    # --- Extract ID and Determine Output Filename ---
    # Use regular expression to find the number part in 'hepaticvessel_XXXXX.nii.gz'
    match = re.search(r'hepaticvessel_(\d+)\.nii\.gz', filename)
    if not match:
        print(f"\nWarning: Could not extract numeric ID from filename '{filename}'. Skipping this file.")
        continue # Skip to the next file if the name format is unexpected

    # Extract the number (as integer)
    file_id = int(match.group(1))

    # Format the output filename as 'setXX.nii' (e.g., set03.nii, set10.nii)
    # Using :02d formats the integer with leading zero if needed (e.g., 3 -> 03)
    # Adjust formatting if test set uses different padding (e.g., :03d for set003)
    output_filename = f"set{file_id:02d}.nii"

    # Define the full paths for the output files for both tasks
    output_task1_path = os.path.join(task1_folder, output_filename)
    output_task2_path = os.path.join(task2_folder, output_filename)

    try:
        # --- Load Input Image and Metadata ---
        # Read the input prediction file using SimpleITK
        input_sitk = sitk.ReadImage(input_filepath)

        # Store original metadata (spacing, origin, direction) to apply to output files
        # This ensures the output files have the same geometry as the input
        original_spacing = input_sitk.GetSpacing()     # (x, y, z) order
        original_origin = input_sitk.GetOrigin()
        original_direction = input_sitk.GetDirection()

        # --- Process for Task 2 (Classification) ---
        # Get the image data as a NumPy array (z, y, x order)
        task2_array = sitk.GetArrayFromImage(input_sitk)

        # **Important Assumption:** The input array already contains the correct
        # classification labels (e.g., 0 for background, 1 for class A, 2 for class B, etc.)
        # as required by the (missing) Table 1 for Task 2.
        # Ensure the array has an integer type appropriate for labels. uint8 is common.
        task2_array = task2_array.astype(np.uint8) # Or np.int16, etc., if needed

        # Create a new SimpleITK image from the Task 2 NumPy array
        task2_sitk_out = sitk.GetImageFromArray(task2_array)

        # Apply the original metadata to the Task 2 output image
        task2_sitk_out.SetSpacing(original_spacing)
        task2_sitk_out.SetOrigin(original_origin)
        task2_sitk_out.SetDirection(original_direction)

        # Save the Task 2 image as a .nii file (uncompressed)
        sitk.WriteImage(task2_sitk_out, output_task2_path, useCompression=False)

        # --- Process for Task 1 (Segmentation - Centerline Map) ---
        # The centerline map code requires a binary input mask (vessel vs. non-vessel)
        # Create this binary mask from the Task 2 classification array.
        # **Assumption:** Any non-zero label in Task 2 corresponds to a vessel.
        # Modify this logic if Table 1 specifies differently (e.g., only label 1 is vessel).
        task1_input_mask = (task2_array > 0).astype(np.uint8)

        # Get spacing in (z, y, x) order, required by scipy's distance_transform_edt
        spacing_zyx = original_spacing[::-1]

        # Generate the centerline map using the provided function
        # The result is a float32 array normalized between 0 and 1 within the mask
        task1_array_float = make_centerline_map(task1_input_mask, spacing=spacing_zyx)

        # Ensure the output array is float32
        task1_array_float = task1_array_float.astype(np.float32)

        # Create a new SimpleITK image from the Task 1 NumPy array
        task1_sitk_out = sitk.GetImageFromArray(task1_array_float)

        # Apply the original metadata to the Task 1 output image
        task1_sitk_out.SetSpacing(original_spacing)
        task1_sitk_out.SetOrigin(original_origin)
        task1_sitk_out.SetDirection(original_direction)

        # Save the Task 1 image as a .nii file (uncompressed)
        sitk.WriteImage(task1_sitk_out, output_task1_path, useCompression=False)

    except Exception as e:
        # Print an error message if processing fails for a specific file
        print(f"\nError processing file '{filename}': {e}")
        # Optionally, add more detailed error logging here

# --- Final Confirmation ---
print("\n------------------------------------")
print("Processing complete.")
print(f"Task 1 (Segmentation/Centerline Map) results saved in: {os.path.abspath(task1_folder)}")
print(f"Task 2 (Classification) results saved in: {os.path.abspath(task2_folder)}")
print("\nSubmission Checklist:")
print("- [ ] Ensure the 'Results' folder contains 'Task01' and 'Task02'.")
print("- [ ] Ensure files inside Task01/Task02 are named 'setXX.nii'.")
print("- [ ] Ensure files are uncompressed NIfTI (.nii).")
print("- [ ] Double-check if Task 2 labels match competition requirements (Table 1).")
print("- [ ] Double-check if Task 1 generation logic (binary mask from Task 2) is correct.")
print("- [ ] Create a single .zip or .tar.gz file containing the 'Results' folder.")
print("------------------------------------")

Output folders created/ensured:
- Task 1: /content/drive/MyDrive/Liver Vessel/nnunet/v1.1.1/Results/Task01
- Task 2: /content/drive/MyDrive/Liver Vessel/nnunet/v1.1.1/Results/Task02

Found 20 prediction files to process.
Starting conversion process...


Processing Test Predictions: 100%|██████████| 20/20 [06:52<00:00, 20.63s/it]


------------------------------------
Processing complete.
Task 1 (Segmentation/Centerline Map) results saved in: /content/drive/MyDrive/Liver Vessel/nnunet/v1.1.1/Results/Task01
Task 2 (Classification) results saved in: /content/drive/MyDrive/Liver Vessel/nnunet/v1.1.1/Results/Task02

Submission Checklist:
- [ ] Ensure the 'Results' folder contains 'Task01' and 'Task02'.
- [ ] Ensure files inside Task01/Task02 are named 'setXX.nii'.
- [ ] Ensure files are uncompressed NIfTI (.nii).
- [ ] Double-check if Task 2 labels match competition requirements (Table 1).
- [ ] Double-check if Task 1 generation logic (binary mask from Task 2) is correct.
- [ ] Create a single .zip or .tar.gz file containing the 'Results' folder.
------------------------------------
